# Tarea 5
## Optimización I

### Esteban Reyes Saldaña

In [1]:
import numpy as np
import functions as func

Rosembrock      = func.Rosembrock
Rosembrock_Grad = func.Rosembrock_Grad
Rosembrock_Hess = func.Rosembrock_Hess

Wood            = func.Wood
Wood_Grad       = func.Wood_Grad
Wood_Hess       = func.Wood_Hess

1. Implementa los metodos de Newton y Newton Modificado.


## Método de Netwon

In [2]:
def newton (params = []) :
  # Cargo parámetros
  x_k        = params['x_0']
  x_k_next   = None
  alpha      = params['alpha']
  f          = params['f']
  f_grad     = params['f_grad']
  f_hess     = params['f_hess']
  max_iter   = params['max_iter']
  tau_x      = params['tau_x']
  tau_f      = params['tau_f']
  tau_f_grad = params['tau_grad']
    
  sub_params = {}
        
  # Guardo Parámetros
  f_hist = []
  f_hist.append(f(x_k, params = sub_params))

  g_hist = []
  g_hist.append(np.linalg.norm(f_grad(x_k, params = sub_params)))
    
  # Comienza descenso
  k = 0 
  while True:
    # Calculo Gradiente
    g_k = f_grad(x_k, params = sub_params)
    # Calculo Hessiano
    H_k = f_hess(x_k, params = sub_params)
    # Calculo gradiente
    d_k = np.linalg.solve(H_k, -g_k)      

    # Calculo siguiente valor x_k+1
    x_k_next = x_k + d_k   
        

    # Guardo Parámetros
    f_hist.append(f(x_k_next, sub_params))
    g_hist.append(np.linalg.norm(f_grad(x_k_next, params = sub_params)))

    # Criterios de paro
    if (k > max_iter) :
      break           
    if np.linalg.norm(x_k_next - x_k)/max(np.linalg.norm(x_k), 1.0) < tau_x :
      break              
    if np.abs(f(x_k_next, sub_params) - f(x_k, sub_params)) / max(np.linalg.norm(f(x_k, sub_params)), 1.0) < tau_f :
      break       
    if np.linalg.norm(f_grad(x_k_next, sub_params)) < tau_f_grad :
      break
            
    # Guardo valor anterior   
    x_k = x_k_next       
    k   = k + 1
        
  return np.array(f_hist), np.array(g_hist), x_k_next

## Método de Newton Modificado

In [3]:
def cholesky_plus_identity(A, beta = 1e-3, max_iter = 100):
  diag = np.diag(A)
  min_aii = min(diag)
  if min_aii > 0 :
    tau = 0.0
  else :
    tau = - min_aii + beta

  k = 0
  while k < max_iter :
    A = A + tau * np.eye(diag.shape[0])
    try :
      A = np.linalg.cholesky(A)
    except :
      tau = max(2.0 * tau, beta)
    else :
      k = max_iter
    k += 1

  return A

In [4]:
def newton_modified (params = []) :
    '''
    '''
    # Cargo parámetros
    x_k        = params['x_0']
    x_k_next   = None
    f          = params['f']
    f_grad     = params['f_grad']
    f_hess     = params['f_hess']
    max_iter   = params['max_iter']
    tau_x      = params['tau_x']
    tau_f      = params['tau_f']
    tau_f_grad = params['tau_grad']
    beta       = params['beta']
    max_iter_c = params['cholesky']['max_iter']

    sub_params = {}
        
    # Guardo Parámetros
    f_hist = []
    f_hist.append(f(x_k, params = sub_params))

    g_hist = []
    g_hist.append(np.linalg.norm(f_grad(x_k, params = sub_params)))
              
    # Comienza descenso
    k = 0 
    
    while True:
        # Calculo Gradiente
        g_k = f_grad(x_k, params = sub_params)
        # Calculo Hessiano
        H_k = f_hess(x_k, params = sub_params)
        # Factorizo usando Cholesky Modificado
        L = cholesky_plus_identity(H_k, beta, max_iter_c)   
        # Resuelvo sistema hacia atrás
        y   = np.linalg.solve(L, -g_k) 
        d_k = np.linalg.solve(L.T, y)

        # Calculo siguiente valor x_k+1
        x_k_next = x_k + d_k   
        
        # Guardo Parámetros
        f_hist.append(f(x_k_next, sub_params))
        g_hist.append(np.linalg.norm(f_grad(x_k_next, sub_params)))
        
                  
        # Criterios de paro
        if (k > max_iter) :
            break
            
        if np.linalg.norm(x_k_next - x_k)/max(np.linalg.norm(x_k), 1.0) < tau_x :
            break
                  
        if np.abs(f(x_k_next, sub_params) - f(x_k, sub_params)) / max(np.linalg.norm(f(x_k, sub_params)), 1.0) < tau_f :
            break
        
                  
        if np.linalg.norm(f_grad(x_k_next, sub_params)) < tau_f_grad :
            break
            
        # Guardo valor anterior   
        x_k = x_k_next       
        k   = k + 1
        
    return np.array(f_hist), np.array(g_hist), x_k_next

2. Realice 30 corridas, de los algoritmos de Maximo descenso, Newton y Newton Modificado a las funciones que aparecen abajo. Reporte los promedios
de tiempo de ejecución y número de iteraciones, segun aparece en las tablas
1 y 2. Seleccione los puntos iniciales de modo que cada entrada se obtenga
de la siguiente forma

$$ x_i^0 = x_i^* + \eta $$

donde $ x_i^* $ es la $i$-ésima entrada del óptimo $ x^* $ proporcionado más abajo para cada función y $ \eta \sim \mathcal{U}(-1,1). $

In [5]:
# Métodos para Descenso Gradiente
import optimization as opti
import time as time

back_tracking = opti.back_tracking
bisection     = opti.bisection
des_grad      = opti.des_grad

## Función de Rosembrock



In [7]:
iterations = 30
n = 100

time_backtracking       = []
iterations_backtracking = []

time_newton       = []
iterations_newton = []

time_newton_modified       = []
iterations_newton_modified = []

for i in range(iterations) :
  print('Corrida :', i+1)

  x_0 = [1.0 + np.random.uniform(-1,1) for i in range(n)]
  x_0 = np.array(x_0)
  # Backtracking
  params = {
            'x_0'      : x_0,
            'f'        : Rosembrock,
            'f_grad'   : Rosembrock_Grad,
            'max_iter' : 10000,
            'tau_x'    : 1e-12,
            'tau_f'    : 1e-12,
            'tau_grad' : 1e-12,
            'method'      : 'BackTracking',
            'BackTracking' : {
                                'alpha' : 0.1,
                                'ro'    : 0.1,
                                'c1'    : 1e-4
                             }
          }
  print('Descenso Gradiente...')
  # Mido tiempo
  star                = time.time()
  f_hist, g_hist, x_k = des_grad(params)
  ex_time             = time.time() - star  

  time_backtracking.append(ex_time)
  iterations_backtracking.append(f_hist.shape[0])

  # Newton 
  params = {
            'x_0'      : x_0,
            'f'        : Rosembrock,
            'f_grad'   : Rosembrock_Grad,
            'f_hess'   : Rosembrock_Hess,
            'max_iter' : 10000,
            'tau_x'    : 1e-8,
            'tau_f'    : 1e-8,
            'tau_grad' : 1e-8,
            'alpha'    : 0.5
          }
  print('Newton ...')
  # Mido tiempo
  star                = time.time()
  f_hist, g_hist, x_k = newton(params)
  ex_time             = time.time() - star  

  time_newton.append(ex_time)
  iterations_newton.append(f_hist.shape[0])
    
  # Newton Modificado
  params = {
              'x_0'      : x_0,
              'f'        : Rosembrock,
              'f_grad'   : Rosembrock_Grad,
              'f_hess'   : Rosembrock_Hess,
              'max_iter' : 10000,
              'tau_x'    : 1e-8,
              'tau_f'    : 1e-8,
              'tau_grad' : 1e-8,
              'beta'     : 1e-3,
              'cholesky' : {
                            'max_iter' : 100
                            }
            }
  print('Newton Modificado ...')
  # Mido tiempo
  star                = time.time()
  f_hist, g_hist, x_k = newton_modified(params)
  ex_time             = time.time() - star  

  time_newton_modified.append(ex_time)
  iterations_newton_modified.append(f_hist.shape[0])

    
print('Backtracking:')
print('Tiempo promedio: ', np.mean(time_backtracking), ' Promedio de iteraciones : ', np.mean(iterations_backtracking))
print('Newton:')
print('Tiempo promedio: ', np.mean(time_newton), ' Promedio de iteraciones : ', np.mean(iterations_newton))
print('Newton Modificado:')
print('Tiempo promedio: ', np.mean(time_newton_modified), ' Promedio de iteraciones : ', np.mean(iterations_newton_modified))

Corrida : 1
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 2
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 3
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 4
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 5
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 6
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 7
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 8
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 9
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 10
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 11
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 12
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 13
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 14
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 15
Descenso Gradiente...
Newton ...
Newton Modi

## Función de Wood

In [ ]:
iterations = 30
n = 4

time_backtracking       = []
iterations_backtracking = []

time_newton       = []
iterations_newton = []

time_newton_modified       = []
iterations_newton_modified = []

for i in range(iterations) :
  print('Corrida :', i+1)

  x_0 = [1.0 + np.random.uniform(-1,1) for i in range(n)]
  x_0 = np.array(x_0)
  # Backtracking
  params = {
            'x_0'      : x_0,
            'f'        : Wood,
            'f_grad'   : Wood_Grad,
            'max_iter' : 10000,
            'tau_x'    : 1e-8,
            'tau_f'    : 1e-8,
            'tau_grad' : 1e-8,
            'method'      : 'BackTracking',
            'BackTracking' : {
                                'alpha' : 0.1,
                                'ro'    : 0.1,
                                'c1'    : 1e-4
                             }
          }
  print('Descenso Gradiente...')
  # Mido tiempo
  star                = time.time()
  f_hist, g_hist, x_k = des_grad(params)
  ex_time             = time.time() - star  

  time_backtracking.append(ex_time)
  iterations_backtracking.append(f_hist.shape[0])

  # Newton 
  params = {
            'x_0'      : x_0,
            'f'        : Wood,
            'f_grad'   : Wood_Grad,
            'f_hess'   : Wood_Hess,
            'max_iter' : 10000,
            'tau_x'    : 1e-8,
            'tau_f'    : 1e-8,
            'tau_grad' : 1e-8,
            'alpha'    : 0.5
          }
  print('Newton ...')
  # Mido tiempo
  star                = time.time()
  f_hist, g_hist, x_k = newton(params)
  ex_time             = time.time() - star  

  time_newton.append(ex_time)
  iterations_newton.append(f_hist.shape[0])
    
  # Newton Modificado
  params = {
              'x_0'      : x_0,
              'f'        : Wood,
              'f_grad'   : Wood_Grad,
              'f_hess'   : Wood_Hess,
              'max_iter' : 10000,
              'tau_x'    : 1e-8,
              'tau_f'    : 1e-8,
              'tau_grad' : 1e-8,
              'beta'     : 1e-3,
              'cholesky' : {
                            'max_iter' : 100
                            }
            }
  print('Newton Modificado ...')
  # Mido tiempo
  star                = time.time()
  f_hist, g_hist, x_k = newton_modified(params)
  ex_time             = time.time() - star  

  time_newton_modified.append(ex_time)
  iterations_newton_modified.append(f_hist.shape[0])

    
print('Backtracking:')
print('Tiempo promedio: ', np.mean(time_backtracking), ' Promedio de iteraciones : ', np.mean(iterations_backtracking))
print('Newton:')
print('Tiempo promedio: ', np.mean(time_newton), ' Promedio de iteraciones : ', np.mean(iterations_newton))
print('Newton Modificado:')
print('Tiempo promedio: ', np.mean(time_newton_modified), ' Promedio de iteraciones : ', np.mean(iterations_newton_modified))

Corrida : 1
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 2
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 3
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 4
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 5
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 6
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 7
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 8
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 9
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 10
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 11
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 12
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 13
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 14
Descenso Gradiente...
Newton ...
Newton Modificado ...
Corrida : 15
Descenso Gradiente...
Newton ...
Newton Modi